In [1]:
%config IPCompleter.greedy=True

##### Download in Linux

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O ./tmp/validation-horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O ./tmp/horse-or-human.zip

##### Download in Windows

In [ ]:
!pip install wget

import wget
import os
import zipfile

if not os.path.exists('./tmp'):
    os.mkdir('./tmp')

data_url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'
validation_url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip'
data_save_path = './tmp/horse-or-human.zip'
validation_save_path = './tmp/validation-horse-or-human.zip'
wget.download(data_url, data_save_path)
wget.download(validation_url, validation_save_path)

zip_ref = zipfile.ZipFile(data_save_path, 'r')
zip_ref.extractall('./tmp/horse-or-human')
zip_ref.close()

zip_ref = zipfile.ZipFile(validation_save_path, 'r')
zip_ref.extractall('./tmp/validation-horse-or-human')
zip_ref.close()

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    './tmp/horse-or-human/',
    target_size=(300, 300), 
    batch_size=1, 
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    './tmp/validation-horse-or-human/',
    target_size=(300, 300), 
    batch_size=1, 
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001), metrics=['acc'])
history = model.fit(train_generator, 
                    epochs=15, 
                    verbose=1, 
                    validation_data=validation_generator, 
                    validation_steps=8)

##### **利用keras-tuner寻找最优参数**
该库实际使用中有问题，或与tensorflow版本相关

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    './tmp/horse-or-human/',
    target_size=(150,150), 
    batch_size=32, 
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    './tmp/validation-horse-or-human/',
    target_size=(150,150), 
    batch_size=32, 
    class_mode='binary'
)

hp = HyperParameters()
def build_model(hp):
    model = tf.keras.models.Sequential()        
    model.add(tf.keras.layers.Conv2D(hp.Choice('num_filters_top_layer',values=[16,64],default=16), (3,3), 
                                     activation='relu', input_shape=(150, 150, 3)))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    for i in range(hp.Int("num_conv_layers",1,3)):
        model.add(tf.keras.layers.Conv2D(hp.Choice(f'num_filters_layer{i}',values=[16,64],default=16), (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hp.Int("hidden_units",128,512,step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])
    return model

In [ ]:
tuner=Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=10,
    directory='horse_human_params',
    hyperparameters=hp,
    project_name='my_horse_human_project'
)
tuner.search(train_generator,epochs=10,validation_data=validation_generator)

In [ ]:
best_hps=tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)
model=tuner.hypermodel.build(best_hps)
model.summary()